In [1]:
%config IPCompleter.greedy = True  
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

from seaborn import countplot
from matplotlib.pyplot import figure, show

from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.models import FactorRange, ColumnDataSource, TableColumn, DataTable, CustomJS
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler

from bokeh.models.widgets import Panel, Tabs, Select
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models.widgets import CheckboxGroup
from bokeh.layouts import column, row, WidgetBox  
from bokeh.core.properties import value

In [2]:
host = "mysql.nm-interactive.net"
port = 3306
user = "windesheim"
password = "pMjq357Kdee7Sx8C"
database = "windesheim"
client = "mysql"

#Installeer eerst Mysqlcient in Anaconda Navigator
engine = create_engine("{0}://{1}:{2}@{3}:{4}/{5}".format(client, user, password, host, port, database))

datasource = pd.read_sql_query("SELECT * FROM source WHERE KWALIFICATIENIVEAU = 4",engine)
wervingsgebieden = pd.read_sql_query("SELECT * FROM wervingsgebieden",engine)
datasource.info()
wervingsgebieden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169415 entries, 0 to 169414
Data columns (total 14 columns):
nr                    169415 non-null int64
PC4_LEERL             169415 non-null int64
KWALIFICATIENIVEAU    169415 non-null int64
KWALIFICATIE CODE     169415 non-null int64
KWALIFICATIE NAAM     169415 non-null object
MBO SECTOR            169415 non-null object
DOMEIN                169415 non-null object
SECTORUNIT SBB        169415 non-null object
GESLACHT              169415 non-null object
2013   AANT           169415 non-null int64
2014   AANT           169415 non-null int64
2015   AANT           169415 non-null int64
2016   AANT           169415 non-null int64
2017   AANT           169415 non-null int64
dtypes: int64(9), object(5)
memory usage: 18.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 6 columns):
index            44 non-null int64
PostcodeStart    44 non-null int64
PostcodeEind     44 non-null int64
Gemeente  

In [3]:
# Laat n records zien van de datasource
datasource.head()

,nr,PC4_LEERL,KWALIFICATIENIVEAU,KWALIFICATIE CODE,KWALIFICATIE NAAM,MBO SECTOR,DOMEIN,SECTORUNIT SBB,GESLACHT,2013 AANT,2014 AANT,2015 AANT,2016 AANT,2017 AANT
0,1424,1011,4,10388,Horeca-ondernemer/-manager,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,1,0,0,0,0
1,1425,1011,4,10801,Restauratiemedewerker,techniek,Bouw en infra,Sectorunit techniek en gebouwde omgeving,M,1,0,0,0,0
2,1426,1011,4,90038,Artiest (Muziek),techniek,Zorg en welzijn,Sectorunit creatieve industrie en ICT,M,0,0,0,2,0
3,1427,1011,4,90290,Ondernemer detailhandel,economie,Handel en ondernemerschap,Sectorunit handel,M,1,0,0,0,0
4,1428,1011,4,90301,Ondernemer horeca/bakkerij (Manager/ondernemer...,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,0,1,0,0,0


In [4]:
datasource.dtypes

nr                     int64
PC4_LEERL              int64
KWALIFICATIENIVEAU     int64
KWALIFICATIE CODE      int64
KWALIFICATIE NAAM     object
MBO SECTOR            object
DOMEIN                object
SECTORUNIT SBB        object
GESLACHT              object
2013   AANT            int64
2014   AANT            int64
2015   AANT            int64
2016   AANT            int64
2017   AANT            int64
dtype: object

In [5]:
# Laat n records zien van de ingevoerde wervingsgebieden
wervingsgebieden.head()

,index,PostcodeStart,PostcodeEind,Gemeente,Locatie,Categorie
0,0,7940,7999,Meppel,Zwolle,Thuismarkt
1,1,8000,8069,Zwolle,Zwolle,Thuismarkt
2,2,8070,8099,Nunspeet,Zwolle,Thuismarkt
3,3,8100,8159,Raalte,Zwolle,Thuismarkt
4,4,8160,8199,Epe,Zwolle,Thuismarkt


In [6]:
wervingsgebieden.dtypes

index             int64
PostcodeStart     int64
PostcodeEind      int64
Gemeente         object
Locatie          object
Categorie        object
dtype: object

In [7]:
# tijdelijke dataframe - 1000 rows
ds = datasource.sample(n=1000)


In [8]:
# Functie voor het checken van de locatie en categorie, stukje sneller gemaakt ook.
def check_locatie(PostCode):
    test = wervingsgebieden.loc[(wervingsgebieden.PostcodeStart <= PostCode) & (wervingsgebieden.PostcodeEind >= PostCode), ['Locatie', 'Categorie']]
    if test['Locatie'].empty:
        r = pd.Series(['',''])
    else:
        r = pd.Series([test['Locatie'].iloc[0], test['Categorie'].iloc[0]])
    return r

#voeg kolommen toe aan datasource 
datasource[['Locatie', 'Categorie']] = datasource.apply(lambda row: check_locatie(row["PC4_LEERL"]), axis=1)


In [9]:
#Welke soorten zitten er in Locatie?
datasource.Locatie.unique()

#Welke soorten zitten in Categorie?
datasource.Categorie.unique()

array(['Aangrenzend Primair', '', 'Thuismarkt', 'Aangrenzend Secundair'], dtype=object)

In [10]:
datasource.head()

,nr,PC4_LEERL,KWALIFICATIENIVEAU,KWALIFICATIE CODE,KWALIFICATIE NAAM,MBO SECTOR,DOMEIN,SECTORUNIT SBB,GESLACHT,2013 AANT,2014 AANT,2015 AANT,2016 AANT,2017 AANT,Locatie,Categorie
0,1424,1011,4,10388,Horeca-ondernemer/-manager,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,1,0,0,0,0,Flevoland,Aangrenzend Primair
1,1425,1011,4,10801,Restauratiemedewerker,techniek,Bouw en infra,Sectorunit techniek en gebouwde omgeving,M,1,0,0,0,0,Flevoland,Aangrenzend Primair
2,1426,1011,4,90038,Artiest (Muziek),techniek,Zorg en welzijn,Sectorunit creatieve industrie en ICT,M,0,0,0,2,0,Flevoland,Aangrenzend Primair
3,1427,1011,4,90290,Ondernemer detailhandel,economie,Handel en ondernemerschap,Sectorunit handel,M,1,0,0,0,0,Flevoland,Aangrenzend Primair
4,1428,1011,4,90301,Ondernemer horeca/bakkerij (Manager/ondernemer...,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,0,1,0,0,0,Flevoland,Aangrenzend Primair


# 1. Data-Melter

Een punt van aandacht is het feit dat de data niet (mooi) allemaal in gelijke kolommen zitten. Zo zijn bijvoorbeeld de jaargegevens over diverse kolommen verdeeld. Om hiermee goed te kunnen werken, dient deze data onder elkaar te worden gezet.

In [11]:
# Originele dataset behouden, in kopie werken:
dsc = datasource.copy()

In [12]:
# Verkrijg alle 'Jaartal' kolommen (de kolommen die ge-melt dienen te worden)
jaarCols = [col for col in dsc.columns if 'AANT' in col]
jaarCols

['2013   AANT', '2014   AANT', '2015   AANT', '2016   AANT', '2017   AANT']

In [13]:
# Wat zijn dan de id vars? (kolommen die gelijk moeten blijven)
otherCols = [col for col in dsc.columns if not 'AANT' in col]
otherCols

['nr',
 'PC4_LEERL',
 'KWALIFICATIENIVEAU',
 'KWALIFICATIE CODE',
 'KWALIFICATIE NAAM',
 'MBO SECTOR',
 'DOMEIN',
 'SECTORUNIT SBB',
 'GESLACHT',
 'Locatie',
 'Categorie']

In [14]:
# Verander de jaartalkolomnamen naar pure jaartallen (gebruik alleen eerste 4 karakters van kolomnaam)
jaarCleanCols = [str(x)[:4] for x in jaarCols]
jaarCleanCols

['2013', '2014', '2015', '2016', '2017']

In [15]:
# Maak een dictionary voor de rename functie
jaarDict = dict(zip(jaarCols, jaarCleanCols))
jaarDict

{'2013   AANT': '2013',
 '2014   AANT': '2014',
 '2015   AANT': '2015',
 '2016   AANT': '2016',
 '2017   AANT': '2017'}

In [16]:
# Rename de betreffende colommen
dsc = dsc.rename(columns=jaarDict)

In [17]:
# Voer werkelijke melt uit
dscMelt = pd.melt(dsc, id_vars=otherCols, var_name="Jaar", value_name="Aantal")

In [18]:
dscMelt.head()

,nr,PC4_LEERL,KWALIFICATIENIVEAU,KWALIFICATIE CODE,KWALIFICATIE NAAM,MBO SECTOR,DOMEIN,SECTORUNIT SBB,GESLACHT,Locatie,Categorie,Jaar,Aantal
0,1424,1011,4,10388,Horeca-ondernemer/-manager,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,Flevoland,Aangrenzend Primair,2013,1
1,1425,1011,4,10801,Restauratiemedewerker,techniek,Bouw en infra,Sectorunit techniek en gebouwde omgeving,M,Flevoland,Aangrenzend Primair,2013,1
2,1426,1011,4,90038,Artiest (Muziek),techniek,Zorg en welzijn,Sectorunit creatieve industrie en ICT,M,Flevoland,Aangrenzend Primair,2013,0
3,1427,1011,4,90290,Ondernemer detailhandel,economie,Handel en ondernemerschap,Sectorunit handel,M,Flevoland,Aangrenzend Primair,2013,1
4,1428,1011,4,90301,Ondernemer horeca/bakkerij (Manager/ondernemer...,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,Flevoland,Aangrenzend Primair,2013,0


In [19]:
dscMelt['Jaar'].unique()

array(['2013', '2014', '2015', '2016', '2017'], dtype=object)

In [20]:
dscMelt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847075 entries, 0 to 847074
Data columns (total 13 columns):
nr                    847075 non-null int64
PC4_LEERL             847075 non-null int64
KWALIFICATIENIVEAU    847075 non-null int64
KWALIFICATIE CODE     847075 non-null int64
KWALIFICATIE NAAM     847075 non-null object
MBO SECTOR            847075 non-null object
DOMEIN                847075 non-null object
SECTORUNIT SBB        847075 non-null object
GESLACHT              847075 non-null object
Locatie               847075 non-null object
Categorie             847075 non-null object
Jaar                  847075 non-null object
Aantal                847075 non-null int64
dtypes: int64(5), object(8)
memory usage: 84.0+ MB


# 2. Voorbeeld in Bokeh: Dataframe SBB - Jaar

Poging een voorbeeld te maken in Bokeh

In [21]:
# maak document, met daarin grafiek. Hier zullen (uiteindelijk) meerdere grafieken moeten worden 
# weergegeven met meerdere opties.
df = dscMelt.copy()

# defineer variabelen voor grafiek
df_totaalSBB = df.groupby(['SECTORUNIT SBB', 'Jaar'], as_index=False).sum()[['SECTORUNIT SBB', 'Jaar', 'Aantal']]
df_totaalSBB.head()

# maak document, met daarin grafiek. Hier zullen (uiteindelijk) meerdere grafieken moeten worden 
# weergegeven met meerdere opties.
dx = dscMelt.copy()

# defineer variabelen voor grafiek
dx_totaalSBB = dx.groupby(['SECTORUNIT SBB', 'Jaar']).sum()[['Aantal']]
dx_totaalSBB.head()

Aantal
SECTORUNIT SBB Jaar        
Bovensectoraal 2013     369
               2014     517
               2015     433
               2016     306
               2017     562

In [22]:
df_totaalSBB.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 0 to 44
Data columns (total 3 columns):
SECTORUNIT SBB    45 non-null object
Jaar              45 non-null object
Aantal            45 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.4+ KB


In [23]:
#Defineer brondata & kolommen
source = ColumnDataSource(df_totaalSBB)
original_source = ColumnDataSource(df_totaalSBB)
columns = [
   TableColumn(field="SECTORUNIT SBB", title="Sector"),
   TableColumn(field="Jaar", title="Jaar"),
   TableColumn(field="Aantal", title="Aantal"),
]

In [24]:
df_totaalSBB.describe()

,Aantal
count,45.000000
mean,7048.911111
std,6181.859649
min,306.000000
25%,2472.000000
50%,6310.000000
75%,9214.000000
max,22298.000000


##  Voobeeldtabel met filter Sectorunit SBB - Jaar
Hieronder een voorbeeld. 
1. Ga eerst opnieuw een dataframe aanmaken (zie 2. Voorbeeld in Bokeh: Poging een voorbeeld te maken in Bokeh)
2. Pas daarna onderstaande aan
  - Definieer opnieuw brondata (df_..ietsnieuws..) & kolommen
  - Pas de combined callback code aan, je hebt tenslotte een nieuw dataframe met     eigen kolomnamen

In [34]:
# Voorbeeld datatable output voor alle datatabellen.
from bokeh.io import curdoc
output_notebook()

#Defineer brondata & kolommen
source = ColumnDataSource(df_totaalSBB)
original_source = ColumnDataSource(df_totaalSBB)
columns = [
   TableColumn(field="SECTORUNIT SBB", title="Sector"),
   TableColumn(field="Jaar", title="Jaar"),
   TableColumn(field="Aantal", title="Aantal"),
]

#maak data tabel
data_table = DataTable(source=source, columns=columns)

# callback code to be used by all the filter widgets
combined_callback_code = """
var data = source.data;
var original_data = original_source.data;
var sector = sector_select_obj.value;
console.log("sector: " + sector);
var jaar = jaar_select_obj.value;
console.log("jaar: " + jaar);
for (var key in original_data) {
    data[key] = [];
    for (var i = 0; i < original_data['SECTORUNIT SBB'].length; ++i) {
        if ((sector === "ALL" || original_data['SECTORUNIT SBB'][i] === sector) &&
            (jaar === "ALL" || original_data['Jaar'][i] === jaar)) {
            data[key].push(original_data[key][i]);
        }
    }
}
target_obj.change.emit();;
source.change.emit();
"""

# define the filter widgets, without callbacks for now
sector_list = ['ALL'] + df_totaalSBB['SECTORUNIT SBB'].unique().tolist()
sector_select = Select(title="Sector:", value=sector_list[0], options=sector_list)
jaar_list = ['ALL'] + df_totaalSBB['Jaar'].unique().tolist()
jaar_select = Select(title="Jaar:", value=jaar_list[0], options=jaar_list)

# now define the callback objects now that the filter widgets exist
generic_callback = CustomJS(
    args=dict(source=source, 
              original_source=original_source, 
              sector_select_obj=sector_select, 
              jaar_select_obj=jaar_select, 
              target_obj=data_table),
    code=combined_callback_code
)

# finally, connect the callbacks to the filter widgets
sector_select.js_on_change('value', generic_callback)
jaar_select.js_on_change('value', generic_callback)

#  
def make_plot(df_totaalSBB):
    plot = figure(plot_height=400,title="Totalen per SBB sector per jaar")
    plot.vbar(x='SECTORUNIT SBB', top='Aantal', width=0.8, source=source)
    plot.width=900
    plot.y_range.start = 0
    plot.x_range.range_padding = 0.1
    plot.xaxis.major_label_orientation = 1
    return plot

def update_plot(attr, old, new):
    x = sector_select.js_on_change('value', generic_callback, update_plot)
    Aantal = jaar_select.js_on_change('value', generic_callback, update_plot)
    make_plot(df_totaalSBB)

# plot it 
plot = make_plot(df_totaalSBB) 
#update values usinsg widgets 
sector_select.js_on_change('value', generic_callback, update_plot)
jaar_select.js_on_change('value', generic_callback, update_plot)
# picture the layout of plot and controls on ui 
controls = column(sector_select,jaar_select)
# create a layout instance containing the complete app layout
layout = column(controls,data_table,plot)
curdoc().add_root(layout)

#doc.title = "Sectorunit SBB chart test"
#doc.add_root(p)
show(p)


Loading BokehJS ...

ValueError: not all callback values are CustomJS instances

## Updaten van bar plot (callback) nav gewijzigde selectie tabel

In [26]:
df.head()

,nr,PC4_LEERL,KWALIFICATIENIVEAU,KWALIFICATIE CODE,KWALIFICATIE NAAM,MBO SECTOR,DOMEIN,SECTORUNIT SBB,GESLACHT,Locatie,Categorie,Jaar,Aantal
0,1424,1011,4,10388,Horeca-ondernemer/-manager,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,Flevoland,Aangrenzend Primair,2013,1
1,1425,1011,4,10801,Restauratiemedewerker,techniek,Bouw en infra,Sectorunit techniek en gebouwde omgeving,M,Flevoland,Aangrenzend Primair,2013,1
2,1426,1011,4,90038,Artiest (Muziek),techniek,Zorg en welzijn,Sectorunit creatieve industrie en ICT,M,Flevoland,Aangrenzend Primair,2013,0
3,1427,1011,4,90290,Ondernemer detailhandel,economie,Handel en ondernemerschap,Sectorunit handel,M,Flevoland,Aangrenzend Primair,2013,1
4,1428,1011,4,90301,Ondernemer horeca/bakkerij (Manager/ondernemer...,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,Flevoland,Aangrenzend Primair,2013,0


In [27]:
#from bokeh.server.server import Server
#from bokeh.application import Application
#from bokeh.application.handlers.function import FunctionHandler

#apps = {'/': Application(FunctionHandler(make_document))}

#server = Server(apps, port=5006, allow_websocket_origin=['localhost:5006','nm-interactive.net:5006'])
#server.start()

In [28]:
#server.stop()


In [29]:
#source.column_names

In [30]:
df.head()

,nr,PC4_LEERL,KWALIFICATIENIVEAU,KWALIFICATIE CODE,KWALIFICATIE NAAM,MBO SECTOR,DOMEIN,SECTORUNIT SBB,GESLACHT,Locatie,Categorie,Jaar,Aantal
0,1424,1011,4,10388,Horeca-ondernemer/-manager,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,Flevoland,Aangrenzend Primair,2013,1
1,1425,1011,4,10801,Restauratiemedewerker,techniek,Bouw en infra,Sectorunit techniek en gebouwde omgeving,M,Flevoland,Aangrenzend Primair,2013,1
2,1426,1011,4,90038,Artiest (Muziek),techniek,Zorg en welzijn,Sectorunit creatieve industrie en ICT,M,Flevoland,Aangrenzend Primair,2013,0
3,1427,1011,4,90290,Ondernemer detailhandel,economie,Handel en ondernemerschap,Sectorunit handel,M,Flevoland,Aangrenzend Primair,2013,1
4,1428,1011,4,90301,Ondernemer horeca/bakkerij (Manager/ondernemer...,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,Flevoland,Aangrenzend Primair,2013,0


## 1. DF Waar woonachtig naar mbo opleiding in absolute aantallen 

In [31]:
# maak document, met daarin grafiek. Hier zullen (uiteindelijk) meerdere grafieken moeten worden 
# weergegeven met meerdere opties.
df = dscMelt.copy()
# defineer variabelen voor grafiek
df_opleid = df.groupby(['PC4_LEERL','Locatie','KWALIFICATIE NAAM', 'Jaar'], as_index=False).sum()[['PC4_LEERL','Locatie','KWALIFICATIE NAAM', 'Jaar', 'Aantal']]

# maak document, met daarin grafiek. Hier zullen (uiteindelijk) meerdere grafieken moeten worden 
# weergegeven met meerdere opties.
dr = dscMelt.copy()

# defineer variabelen voor grafiek
dr_opleid = dr.groupby(['PC4_LEERL','Locatie','KWALIFICATIE NAAM', 'Jaar']).sum()[['Aantal']]
dr_opleid.head()

Aantal
PC4_LEERL Locatie   KWALIFICATIE NAAM   Jaar        
1011      Flevoland Apothekersassistent 2013       0
                                        2014       0
                                        2015       0
                                        2016       0
                                        2017       1

In [32]:
df_opleid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 721225 entries, 0 to 721224
Data columns (total 5 columns):
PC4_LEERL            721225 non-null int64
Locatie              721225 non-null object
KWALIFICATIE NAAM    721225 non-null object
Jaar                 721225 non-null object
Aantal               721225 non-null int64
dtypes: int64(2), object(3)
memory usage: 33.0+ MB


##  Tabel met filter Woonachtig - MBO opleiding
  - Definieer opnieuw brondata (df_..ietsnieuws..) & kolommen
  - Pas de combined callback code aan, je hebt tenslotte een nieuw dataframe met     eigen kolomnamen

In [33]:
# Voorbeeld datatable output voor alle datatabellen.
output_notebook()

#Defineer brondata & kolommen
source = ColumnDataSource(dr_opleid)
original_source = ColumnDataSource(dr_opleid)
columns = [
   TableColumn(field="PC4_LEERL", title="Postcode"),
   TableColumn(field="Locatie", title="Wervingsgebied"),
   TableColumn(field="KWALIFICATIE NAAM", title="Opleiding"),
   TableColumn(field="Jaar", title="Jaar"),
   TableColumn(field="Aantal", title="Aantal"), 
]

#maak data tabel
data_table = DataTable(source=source, columns=columns)

# callback code to be used by all the filter widgets
combined_callback_code = """
var data = source.data;
var original_data = original_source.data;
var postcode = postcode_select_obj.value;
console.log("postcode: " + postcode);
var wervingsgebied = wervingsgebied_select_obj.value;
console.log("wervingsgebied: " + wervingsgebied);
var opleiding = opleiding_select_obj.value;
console.log("opleiding: " + opleiding);
var jaar = jaar_select_obj.value;
console.log("jaar: " + jaar);
for (var key in original_data) {
    data[key] = [];
    for (var i = 0; i < original_data['PC4_LEERL'].length; ++i) {
        if ((postcode === "ALL" || original_data['PC4_LEERL'][i] === postcode) &&
            (wervingsgebied === "ALL" || original_data['Locatie'][i] === wervingsgebied) &&
            (opleiding === "ALL" || original_data['KWALIFICATIE NAAM'][i] === opleiding) &&
            (jaar === "ALL" || original_data['Jaar'][i] === jaar)) {
            data[key].push(original_data[key][i]);
        }
    }
}
target_obj.change.emit();;
source.change.emit();
"""

# define the filter widgets, without callbacks for now
postcode_list = ['ALL'] + dr_opleid['PC4_LEERL'].unique().tolist()
postcode_select = Select(title="Postcode:", value=postcode_list[0], options=postcode_list)
wervingsgebied_list = ['ALL'] + dr_opleid['Locatie'].unique().tolist()
wervingsgebied_select = Select(title="Wervingsgebied:", value=wervingsgebied_list[0], options=wervingsgebied_list)
opleiding_list = ['ALL'] + dr_opleid['KWALIFICATIE NAAM'].unique().tolist()
opleiding_select = Select(title="Opleiding:", value=opleiding_list[0], options=opleiding_list)
jaar_list = ['ALL'] + dr_opleid['Jaar'].unique().tolist()
jaar_select = Select(title="Jaar:", value=jaar_list[0], options=jaar_list)

# now define the callback objects now that the filter widgets exist
generic_callback = CustomJS(
    args=dict(source=source, 
              original_source=original_source, 
              postcode_select_obj=postcode_select,
              wervingsgebied_select_obj=wervingsgebied_select,
              opleiding_select_obj=opleiding_select,
              jaar_select_obj=jaar_select, 
              target_obj=data_table),
    code=combined_callback_code
)

# finally, connect the callbacks to the filter widgets
postcode_select.js_on_change('value', generic_callback)
wervingsgebied_select.js_on_change('value', generic_callback)
opleiding_select.js_on_change('value', generic_callback)
jaar_select.js_on_change('value', generic_callback)


#layout = row(p, jaren)
p = column(postcode_select,wervingsgebied_select,opleiding_select,jaar_select,data_table)

#doc.title = "Waar gediplomeerde mbo-4 woonachtig naar mbo-opleiding in absolute aantallen"
#doc.add_root(p)
show(p)

Loading BokehJS ...

KeyError: 'PC4_LEERL'